In [9]:
import requests

In [10]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [11]:
data = requests.get(standings_url)


In [12]:
from bs4 import BeautifulSoup

In [13]:
soup = BeautifulSoup(data.text)


In [14]:
standings_table = soup.select('table.stats_table')[0]


In [15]:
links = standings_table.find_all('a')

In [ ]:
links[0].get("href")

In [ ]:
links = [l.get("href") for l in links]


In [ ]:
team_urls = [f"http://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [18]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
matches[0].head()

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and "all_comps/shooting/" in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK","PKatt"]], on="Date")

In [ ]:
team_data.head()


In [ ]:
matches[0].shape


In [ ]:
shooting.shape

In [1]:
years = list(range(2024, 2020, -1))

In [2]:
years

[2024, 2023, 2022, 2021]

In [3]:
all_matches = []

In [4]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [19]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url= f"https://fbref.com/{previous_season}"
    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace('-Stats', "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK","PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
    
    

In [24]:
match_df = pd.concat(all_matches)


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1,1,2024,ManchesterCity
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0,0,2024,ManchesterCity
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0,0,2024,ManchesterCity
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0,0,2024,ManchesterCity
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0,0,2024,ManchesterCity


In [23]:
match_df.columns = [c.lower() for c in match_df.columns]

AttributeError: 'builtin_function_or_method' object has no attribute 'lower'

In [22]:
match_df.to_csv("../data-scrapped/matches.csv")